In [17]:
from guardrails.hub import ToxicLanguage, CompetitorCheck
from guardrails import Guard
import requests
import json

In this example, we will apply guardrails to an image generation model

In [18]:
# Here we used 2 guardrails :
# Toxic Language check and Competitors check
guard = Guard().use(
    ToxicLanguage, threshold=0.5, validation_method="sentence", on_fail="exception"
).use(CompetitorCheck, ["Apple", "Samsung"], "exception")

In [19]:
# Importing the Hugging face API token from config.json file
def load_hugging_face_token():
    with open('config.json') as config_file:
        config = json.load(config_file)
        return config['hugging_face_token']

In [20]:
# The url for the hosted model and the header files for the http request
API_URL = "https://api-inference.huggingface.co/models/stabilityai/stable-diffusion-xl-base-1.0"
headers = {"Authorization": f"Bearer {load_hugging_face_token()}"}

In [22]:
# creating a http request and then returning the response
def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.content

In [23]:
# creating a wrapper for the above function which takes in the input and returns the output
# The data type of this function is bytes
def ai_model(user_input):
    payload = {
        "inputs" : user_input,
    }
    ai_output = query(payload)
    return ai_output

In [24]:
# First we validate the input and then return True or False depending on whether the input passes the required conditions or not.
def guardrails_ai_wrapper(user_input):
    try:
        guard.validate(
            user_input
        )

        return True

    except Exception as e:
        print(str(e))
        return False

In [25]:
# collecting the input from the user
prompt = input("Enter the text for image generation : ")

In [26]:
# importing the required libraries to convert bytes into image and then output it
import io
from PIL import Image

In [27]:
# function to convert bytes to image
def get_image(user_input) -> Image:
    image_bytes = ai_model(user_input)
    image = Image.open(io.BytesIO(image_bytes))
    return image

In [28]:
# if the user input passes the required conditions, then output the image
if guardrails_ai_wrapper(prompt):
    get_image(prompt).show()